In [ ]:
!pip install haversine

In [ ]:
import pandas as pd
import plotly.express as px
import folium
import re
from haversine import haversine
import plotly.graph_objects as go
import numpy as np

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
#backup
df1 = df.copy()

#1. convertendo a coluna Age de texto para número
linhas_selecionadas = (df1['Delivery_person_Age'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()

df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)
df1.shape

#2 convertendo a coluna Ratings de texto para numero decimal (float)
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype(float)

#3 convertendo a coluna order_date de texto para data
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format='%d-%m-%Y')

#4 convertendo multiple_deliveries de texto para numero inteiro (int)
linhas_selecionadas2 = (df1['multiple_deliveries'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas2, :].copy()
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype(int)

#5Removendo os espaços dentro de strings/texto/object
df1.loc[:, "ID"] = df1.loc[:, "ID"].str.strip()
df1.loc[:, "Road_traffic_density"] = df1.loc[:, "Road_traffic_density"].str.strip()
df1.loc[:, "Type_of_order"] = df1.loc[:, "Type_of_order"].str.strip()
df1.loc[:, "Type_of_vehicle"] = df1.loc[:, "Type_of_vehicle"].str.strip()
df1.loc[:, "City"] = df1.loc[:, "City"].str.strip()
df1.loc[:, "Festival"] = df1.loc[:, "Festival"].str.strip()

#6 Removendo os NaN
df1 = df1.loc[df1['Road_traffic_density'] != 'NaN', :]
df1 = df1.loc[df1['City'] != 'NaN', :]
df1 = df1.loc[df1['Time_taken(min)'] != 'NaN', :]
df1 = df1.loc[df1['Festival'] != 'NaN', :]

#7 Removendo o (min) do Time_Taken
df1['Time_taken(min)'] = df1['Time_taken(min)'].apply(lambda x: x.split( '(min)')[1])
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)

In [ ]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30


#VISÃO DA EMPRESA

##1. Quantidade pedidos por dia.

In [ ]:
cols = ['ID', 'Order_Date']

df_aux = df1.loc[:, cols].groupby('Order_Date').count().reset_index()
df_aux.head()

,Order_Date,ID
0,2022-02-11,904
1,2022-02-12,794
2,2022-02-13,872
3,2022-02-14,790
4,2022-02-15,860


In [ ]:
px.bar(df_aux, x='Order_Date', y='ID')

##2. Quantidade de pedidos por semana.

In [ ]:
#criar a coluna de semana convertendo as datas
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U') #w se for para começar na segunda
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24,11
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33,12
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26,11
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21,14
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30,12


In [ ]:
df_aux2 = df1.loc[:, ["ID", 'week_of_year']].groupby('week_of_year').count().reset_index()
df_aux2.head()

,week_of_year,ID
0,06,1698
1,07,4964
2,09,4974
3,10,6723
4,11,6879


In [ ]:
px.line(df_aux2, x='week_of_year', y='ID')

##3. Distribuição dos pedidos por tipo de tráfego

In [ ]:
df_aux3 = df1.loc[:, ['ID', 'Road_traffic_density']].groupby('Road_traffic_density').count().reset_index()

df_aux3['entrega_perc'] = df_aux3['ID'] / df_aux3['ID'].sum()
df_aux3

,Road_traffic_density,ID,entrega_perc
0,High,4119,0.098988
1,Jam,13207,0.317392
2,Low,14112,0.339141
3,Medium,10173,0.244479


In [ ]:
px.pie(df_aux3, values='entrega_perc', names='Road_traffic_density')

##4. comparação do volume de pedidos por cidade e tipo de tráfico

In [ ]:
df_aux4 = df1.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()
df_aux4

,City,Road_traffic_density,ID
0,Metropolitian,High,3215
1,Metropolitian,Jam,10581
2,Metropolitian,Low,10254
3,Metropolitian,Medium,7917
4,Semi-Urban,High,16
5,Semi-Urban,Jam,126
6,Semi-Urban,Medium,10
7,Urban,High,888
8,Urban,Jam,2500
9,Urban,Low,3858


In [ ]:
px.scatter(df_aux4, x='City', y='Road_traffic_density', size='ID', color='City')

##5. A quantidade de pedidos por entregador por semana

In [ ]:
df_aux5 = df1.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()

df_aux5

,week_of_year,ID
0,06,1698
1,07,4964
2,09,4974
3,10,6723
4,11,6879
5,12,5831
6,13,6688
7,14,3854


In [ ]:
df_aux6 = df1.loc[:,['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()
df_aux6

,week_of_year,Delivery_person_ID
0,06,598
1,07,600
2,09,720
3,10,720
4,11,720
5,12,720
6,13,720
7,14,719


In [ ]:
df_aux_u = pd.merge(df_aux5, df_aux6, how='inner')
df_aux_u['order_by_deliver'] = df_aux_u['ID'] / df_aux_u['Delivery_person_ID']
df_aux_u

,week_of_year,ID,Delivery_person_ID,order_by_deliver
0,06,1698,598,2.839465
1,07,4964,600,8.273333
2,09,4974,720,6.908333
3,10,6723,720,9.337500
4,11,6879,720,9.554167
5,12,5831,720,8.098611
6,13,6688,720,9.288889
7,14,3854,719,5.360223


In [ ]:
px.line(df_aux_u, x='week_of_year', y='order_by_deliver')

##6. A localização central de cada cidade por tipo de tráfego

In [ ]:
df_aux7 = df1.loc[:, ["City", 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']].groupby(['City', 'Road_traffic_density']).median().reset_index()
df_aux7 = df_aux7.head()
df_aux7

,City,Road_traffic_density,Delivery_location_latitude,Delivery_location_longitude
0,Metropolitian,High,18.574080,76.305001
1,Metropolitian,Jam,18.661440,76.017648
2,Metropolitian,Low,18.629156,75.993365
3,Metropolitian,Medium,18.644080,76.017648
4,Semi-Urban,High,20.463477,76.814977


In [ ]:
map = folium.Map()
map

In [ ]:
for index, location_info in df_aux7.iterrows():
    folium.Marker(
        [location_info['Delivery_location_latitude'], location_info['Delivery_location_longitude']],
        popup=f"City: {location_info['City']}, Traffic: {location_info['Road_traffic_density']}"
    ).add_to(map)


In [ ]:
map

#VISÃO ENTREGADORES

## 1. A menor e maior idade dos entregadores

In [ ]:
# A maior idade dos entregadores
max_idade = df1.loc[:, 'Delivery_person_Age'].max()
print(f'A maior idade dos entregadores é de {max_idade}')

A maior idade dos entregadores é de 39


In [ ]:
# A menor idade dos entregadores
min_idade = df1.loc[:, 'Delivery_person_Age'].min()
print(f'A menor idade dos entregadores é de {min_idade}')

A menor idade dos entregadores é de 20


## 2. A pior e melhor condição de veículos

In [ ]:
# A melhor condição dos veículos
melhor = df1.loc[:, 'Vehicle_condition'].max()
print(f'A melhor condição dos veículos é de {melhor}')

A melhor condição dos veículos é de 2


In [ ]:
# A pior condição dos veículos
pior = df1.loc[:, 'Vehicle_condition'].min()
print(f'A pior condição dos veículos é de {pior}')

A pior condição dos veículos é de 0


## 3. Avaliação média por entregador

In [ ]:
media = df1.loc[:, ["Delivery_person_ID", "Delivery_person_Ratings"]].groupby("Delivery_person_ID").mean().reset_index()
media

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000
...,...,...
1315,VADRES19DEL02,4.630357
1316,VADRES19DEL03,4.670270
1317,VADRES20DEL01,4.620370
1318,VADRES20DEL02,4.591111


## 4. A avaliação média e desvio de padrão por tipo de tráfego

In [ ]:
media_e_desvio_trafego = df1.loc[:, ['Delivery_person_Ratings', 'Road_traffic_density']].groupby('Road_traffic_density').agg(['mean', 'std']).reset_index()
media_e_desvio_trafego.columns = ['Traffic', 'Delivery_mean', 'Delivery_std']
media_e_desvio_trafego

,Traffic,Delivery_mean,Delivery_std
0,High,4.652538,0.272699
1,Jam,4.594428,0.329531
2,Low,4.645947,0.337211
3,Medium,4.660335,0.273956


## 5. A avaliação média e desvio padrão por condições climáticas.

In [ ]:
media_e_desvio_clima = df1.loc[:, ['Delivery_person_Ratings', 'Weatherconditions']].groupby('Weatherconditions').agg(['mean', 'std']).reset_index()
media_e_desvio_clima.columns = ['Weatherconditions', 'Delivery_mean', 'Delivery_std']
media_e_desvio_clima

,Weatherconditions,Delivery_mean,Delivery_std
0,conditions Cloudy,4.651945,0.281016
1,conditions Fog,4.653087,0.274974
2,conditions Sandstorms,4.612144,0.310527
3,conditions Stormy,4.612569,0.312621
4,conditions Sunny,4.656164,0.395057
5,conditions Windy,4.616844,0.304087


## 6. Os 10 entregadores mais rápidos por cidade.

In [ ]:
#USANDO O MAIOR RESULTADO INDIVIDUAL
df2 = df1.loc[:, ['Delivery_person_ID', 'City', 'Time_taken(min)']].groupby(['City','Delivery_person_ID']).min().sort_values(['City', 'Time_taken(min)'], ascending=True).reset_index()

df_a01 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_a02 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_a03 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df_todos1 = pd.concat([df_a01, df_a02, df_a03])
df_todos1

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AGRRES010DEL03,10
1,Metropolitian,AGRRES07DEL03,10
2,Metropolitian,AGRRES12DEL01,10
3,Metropolitian,AGRRES14DEL01,10
4,Metropolitian,AGRRES17DEL03,10
5,Metropolitian,ALHRES08DEL03,10
6,Metropolitian,ALHRES12DEL01,10
7,Metropolitian,ALHRES13DEL01,10
8,Metropolitian,ALHRES14DEL02,10
9,Metropolitian,ALHRES15DEL02,10


In [ ]:
#USANDO A MÉDIA
df2 = df1.loc[:, ['Delivery_person_ID', 'City', 'Time_taken(min)']].groupby(['City','Delivery_person_ID']).mean().sort_values(['City', 'Time_taken(min)'], ascending=True).reset_index()

df_a01 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_a02 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_a03 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df_todos1 = pd.concat([df_a01, df_a02, df_a03])
df_todos1

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,KNPRES03DEL02,15.750000
1,Metropolitian,ALHRES02DEL02,17.800000
2,Metropolitian,KNPRES01DEL01,19.125000
3,Metropolitian,KOLRES01DEL03,19.125000
4,Metropolitian,KOCRES02DEL02,19.250000
5,Metropolitian,KOCRES08DEL03,19.800000
6,Metropolitian,KOCRES16DEL03,20.000000
7,Metropolitian,KOCRES02DEL03,20.375000
8,Metropolitian,GOARES14DEL01,20.400000
9,Metropolitian,DEHRES20DEL03,20.428571


## 7. Os 10 entregadores mais lentos por cidade

In [ ]:
#USANDO O MENOR RESULTADO
df2 = df1.loc[:, ['Delivery_person_ID', 'City', 'Time_taken(min)']].groupby(['City','Delivery_person_ID']).max().sort_values(['City', 'Time_taken(min)'], ascending=False).reset_index()

df_b01 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_b02 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_b03 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df_todos2 = pd.concat([df_b01, df_b02, df_b03])
df_todos2

,City,Delivery_person_ID,Time_taken(min)
1423,Metropolitian,ALHRES010DEL01,54
1424,Metropolitian,AURGRES13DEL01,54
1425,Metropolitian,BANGRES02DEL01,54
1426,Metropolitian,BHPRES16DEL02,54
1427,Metropolitian,CHENRES02DEL02,54
1428,Metropolitian,CHENRES04DEL01,54
1429,Metropolitian,CHENRES07DEL03,54
1430,Metropolitian,CHENRES08DEL01,54
1431,Metropolitian,CHENRES13DEL02,54
1432,Metropolitian,COIMBRES010DEL03,54


In [ ]:
#USANDO A MÉDIA
df2 = df1.loc[:, ['Delivery_person_ID', 'City', 'Time_taken(min)']].groupby(['City','Delivery_person_ID']).mean().sort_values(['City', 'Time_taken(min)'], ascending=False).reset_index()

df_b01 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_b02 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_b03 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df_todos2 = pd.concat([df_b01, df_b02, df_b03])
df_todos2

,City,Delivery_person_ID,Time_taken(min)
1423,Metropolitian,AGRRES02DEL01,39.428571
1424,Metropolitian,AURGRES11DEL03,38.500000
1425,Metropolitian,KOLRES03DEL03,38.142857
1426,Metropolitian,LUDHRES17DEL03,37.000000
1427,Metropolitian,ALHRES18DEL02,36.666667
1428,Metropolitian,GOARES08DEL03,36.000000
1429,Metropolitian,ALHRES12DEL01,35.000000
1430,Metropolitian,AGRRES13DEL02,34.700000
1431,Metropolitian,KNPRES06DEL01,34.555556
1432,Metropolitian,ALHRES010DEL01,34.500000


# VISÃO RESTAURANTES

## 1. A quantidade de entregadores únicos.

In [ ]:
delivery_unique = df1.loc[:, 'Delivery_person_ID'].nunique()
print(f'O numero de entregadores únicos é de: {delivery_unique}')

O numero de entregadores únicos é de: 1320


## 2. A distância média dos restaurantes e dos locais de entrega.

In [ ]:
cols = ['Delivery_location_latitude', 'Delivery_location_longitude', 'Restaurant_latitude', 'Restaurant_longitude']

In [ ]:
df1['distance'] = df1.loc[:, cols].apply(
    lambda x: haversine(
        (x['Delivery_location_latitude'], x['Delivery_location_longitude']),
        (x['Restaurant_latitude'], x['Restaurant_longitude'])
    ),
    axis=1
)

In [ ]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year,distance
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,High,2,Snack,motorcycle,0,No,Urban,24,11,3.025153
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,Jam,2,Snack,scooter,1,No,Metropolitian,33,12,20.183558
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,Low,0,Drinks,motorcycle,1,No,Urban,26,11,1.552760
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21,14,7.790412
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,High,1,Snack,scooter,1,No,Metropolitian,30,12,6.210147


In [ ]:
avg_distance = df1['distance'].mean()
print(f'A distância média das entregas é: {avg_distance:.2f} KM')

A distância média das entregas é: 27.35 KM


In [ ]:
avg_distance = df1.loc[:, ['City', 'distance']].groupby('City').mean().reset_index()
avg_distance

,City,distance
0,Metropolitian,26.697856
1,Semi-Urban,12.875164
2,Urban,29.763272


In [ ]:
fig = go.Figure(data=[go.Pie(labels=avg_distance['City'], values=avg_distance['distance'], pull=[0, 0.1, 0])])
fig.show()

## 3. O tempo médio e o desvio padrão de entrega por cidade.

In [ ]:
cols = ['City', 'Time_taken(min)']

In [ ]:
df_aux7 = df1.loc[:,cols].groupby('City').agg({'Time_taken(min)':['mean', 'std']})
df_aux7.columns = ['avg_time', 'std_time']
df_aux7 = df_aux7.reset_index()
df_aux7

,City,avg_time,std_time
0,Metropolitian,27.428083,9.133374
1,Semi-Urban,49.710526,2.724992
2,Urban,23.209379,8.858049


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(name='Control',
                     x=df_aux7['City'],
                     y=df_aux7['avg_time'],
                     error_y=dict(type='data', array=df_aux7['std_time'])))
fig.update_layout(barmode='group')
fig.show()

## 4. O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido.

In [ ]:
cols = ['City', "Time_taken(min)", 'Type_of_order']

In [ ]:
df_aux8 = df1.loc[:,cols].groupby(['City', 'Type_of_order']).agg({"Time_taken(min)": ['mean', 'std']})
df_aux8.columns = ['avg_time', 'std_time']
df_aux8 = df_aux8.reset_index()
df_aux8

,City,Type_of_order,avg_time,std_time
0,Metropolitian,Buffet,27.252377,9.180606
1,Metropolitian,Drinks,27.275139,9.071588
2,Metropolitian,Meal,27.554270,9.249682
3,Metropolitian,Snack,27.415381,9.152578
4,Semi-Urban,Buffet,49.707317,2.731702
5,Semi-Urban,Drinks,49.625000,2.459347
6,Semi-Urban,Meal,50.300000,3.041665
7,Semi-Urban,Snack,49.408163,2.707385
8,Urban,Buffet,23.449257,9.087620
9,Urban,Drinks,23.157568,8.978597


## 5. O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego.

In [ ]:
cols = ['City', 'Time_taken(min)', 'Road_traffic_density']

In [ ]:
df_aux9 = df1.loc[:,cols].groupby(['City', 'Road_traffic_density']).agg({"Time_taken(min)": ['mean', 'std']})
df_aux9.columns = ['avg_time', 'std_time']
df_aux9 = df_aux9.reset_index()
df_aux9

,City,Road_traffic_density,avg_time,std_time
0,Metropolitian,High,28.103888,7.935947
1,Metropolitian,Jam,31.937908,9.509890
2,Metropolitian,Low,22.193973,6.827415
3,Metropolitian,Medium,27.694076,8.332809
4,Semi-Urban,High,50.125000,2.629956
5,Semi-Urban,Jam,49.841270,2.717095
6,Semi-Urban,Medium,47.400000,2.011080
7,Urban,High,24.206081,8.535574
8,Urban,Jam,27.905200,10.125553
9,Urban,Low,19.336444,6.351539


In [ ]:
fig = px.sunburst(df_aux9, path=['City', 'Road_traffic_density'], values='avg_time',
                  color='std_time', color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df_aux9['std_time']))
fig.show()

## 6. O tempo médio de entrega durante os Festivais.

In [ ]:
cols = ['Time_taken(min)', 'Festival']

In [ ]:
df_aux10 = df1.loc[:, cols].groupby('Festival').agg({'Time_taken(min)': ['mean', 'std']}).reset_index()

df_aux10

Festival Time_taken(min)          
                      mean       std
0       No       26.162741  9.001803
1      Yes       45.518607  4.005399